In [2]:
%pip install aeon --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from aeon.datasets import load_classification
X, y, meta_data = load_classification("FordB", return_metadata=True)
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)

 Shape of X =  (4446, 1, 500)
 Meta data =  {'problemname': 'fordb', 'timestamps': False, 'missing': False, 'univariate': True, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['-1', '1']}


In [3]:
import pandas as pd

df_train = pd.read_csv('FordB_TRAIN.txt', sep='  ', header=None, engine='python')
df_test = pd.read_csv('FordB_TEST.txt', sep='  ', header=None, engine='python')

y_train = df_train[0]
x_train = df_train.drop(0, axis=1)

y_test = df_test[0]
x_test = df_test.drop(0, axis=1)

In [4]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,1.0,-0.731269,-0.331709,0.180212,0.729356,1.226720,1.584864,1.719098,1.622404,1.349615,...,-0.175376,-0.226743,-0.314246,-0.465901,-0.688740,-0.953811,-1.213037,-1.418047,-1.515915,-1.507342
1,-1.0,1.625923,1.174887,0.633557,0.097815,-0.323979,-0.522682,-0.501078,-0.309937,-0.053305,...,0.645025,0.149040,-0.350204,-0.763172,-1.028119,-1.146848,-1.082185,-0.872399,-0.599113,-0.264919
2,-1.0,0.943523,0.928455,0.959547,0.991248,0.996443,0.959983,0.831277,0.580112,0.235417,...,-0.637612,-0.473332,-0.332077,-0.246990,-0.230341,-0.283385,-0.312471,-0.282186,-0.173181,-0.034611
3,-1.0,-0.990329,-1.200994,-1.383529,-1.503270,-1.538072,-1.420004,-1.121234,-0.669089,-0.146006,...,-1.918893,-1.112609,-0.271551,0.495545,1.105130,1.489297,1.608930,1.509580,1.298080,1.038514
4,-1.0,1.014455,0.688760,0.372491,0.081209,-0.188529,-0.464427,-0.759484,-1.068931,-1.362857,...,0.414720,0.661333,0.827912,0.875500,0.809332,0.685075,0.552655,0.467893,0.454164,0.499379


In [5]:
type(x_train)

x_train_np = x_train.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()
x_test_np = x_test.to_numpy()


In [50]:
x_train_np.shape

(3636, 500)

## 1KNN

In [51]:
from aeon.classification.distance_based import (
    ElasticEnsemble,
    KNeighborsTimeSeriesClassifier,
    ProximityForest,
)

from sklearn import metrics


In [ ]:
knn = KNeighborsTimeSeriesClassifier(distance="msm", n_neighbors=1, weights="distance")
knn.fit(x_train_np, y_train_np)
knn_preds = knn.predict(x_test_np)
metrics.accuracy_score(y_test_np, knn_preds)

## MiniRocket

In [7]:
import numpy as np

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [29]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.preprocessing import StandardScaler

from aeon.classification.convolution_based import MiniRocketClassifier
from aeon.datasets import load_arrow_head  # univariate dataset
from aeon.datasets import load_basic_motions  # multivariate dataset
from aeon.regression.convolution_based import MiniRocketRegressor
from aeon.transformations.collection.convolution_based import MiniRocket

In [8]:
### Normalizacion
x_train_n = NormalizeData(x_train_np)
x_test_n = NormalizeData(x_test_np)

In [9]:
x_train_n.shape

(3636, 500)

In [53]:
minirocket = MiniRocket()  
minirocket.fit(x_train_n)

MiniRocket()

In [38]:
X_train_transform = minirocket.transform(x_train_n)
X_train_transform.shape

(3636, 9996)

In [39]:
scaler = StandardScaler(with_mean=False)
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
X_train_scaled_transform = scaler.fit_transform(X_train_transform)
classifier.fit(X_train_scaled_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [40]:
X_test_transform = minirocket.transform(x_test_n)
print(" Score =", classifier.score(X_test_transform, y_test))


 Score = 0.5049382716049383


## MultiRocket con cabeza clasificadora separada

In [55]:
from aeon.transformations.collection.convolution_based import MultiRocket
from sklearn.metrics import accuracy_score

trf = MultiRocket(n_kernels=500)
trf.fit(x_train_n)


MultiRocket(n_kernels=500)

In [ ]:
from sklearn.svm import SVC 

clf = SVC()  
clf.fit(x_train_n, y_train) 

X_test_transform = trf.transform(x_test_n)  

y_pred = clf.predict(X_test_transform)

score = accuracy_score(y_test, y_pred)  

print(f"Score (Precisión): {score}")

## MultiRocket

In [ ]:
import numpy as np

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [35]:
from sklearn.metrics import accuracy_score

from aeon.classification.convolution_based import (
    Arsenal,
    HydraClassifier,
    MiniRocketClassifier,
    MultiRocketClassifier,
    MultiRocketHydraClassifier,
    RocketClassifier,
)

> Normalizacion MultiRocket

In [15]:
x_train_n = NormalizeData(x_train_np)
x_test_n = NormalizeData(x_test_np)

In [16]:
rocket = MultiRocketClassifier(n_kernels=2000)
rocket.fit(x_train_n, y_train)
y_pred = rocket.predict(x_test_n)
accuracy_score(y_test, y_pred)

0.7790123456790123

> Sin Normalizacion MultiRocket

In [23]:
rocket = MultiRocketClassifier(n_kernels=1000, max_dilations_per_kernel=16)
rocket.fit(x_train, y_train)
y_pred = rocket.predict(x_test)
accuracy_score(y_test, y_pred)

0.828395061728395

## Hydra

In [36]:
hydra_clf = HydraClassifier()
hydra_clf.fit(x_train_n, y_train)
y_pred = hydra_clf.predict(x_test)
accuracy_score(y_test, y_pred)

KeyboardInterrupt: 